# Ordering


## Warning: This notebook will place live orders

Use a paper trading account (during market hours).


In [ ]:
from ib_insync import *
import nest_asyncio
nest_asyncio.apply()

util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)
# util.logToConsole()

Create a contract and a market order:

In [ ]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)

order = LimitOrder('SELL', 20000, 1.11)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [ ]:
trade = ib.placeOrder(contract, order)

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [ ]:
ib.sleep(1)
trade.log

``trade`` will also available from ``ib.trades()``:

In [ ]:
assert trade in ib.trades()

Likewise for ``order``:

In [ ]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [ ]:
limitOrder = LimitOrder('BUY', 20000, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

``status`` will change from "PendingSubmit" to "Submitted":

In [ ]:
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

In [ ]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [ ]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

And now cancel it:

In [ ]:
ib.cancelOrder(limitOrder)

In [ ]:
limitTrade.log

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [ ]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

What are our positions?

In [ ]:
ib.positions()

What's the total of commissions paid today?

In [ ]:
sum(fill.commissionReport.commission for fill in ib.fills())

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [ ]:
order = MarketOrder('SELL', 20000)
ib.whatIfOrder(contract, order)

In [ ]:
ib.disconnect()